<a href="https://colab.research.google.com/github/Bayrem-ben/Bayrem-ben.github.io/blob/master/Bayrem_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train = pd.read_table(train_file_path,sep='\t', names=['spam', 'text'])
df_test = pd.read_table(test_file_path,sep='\t', names=['spam', 'text']) 

df_train['spam'] = df_train['spam'].replace({'ham': 0, 'spam': 1})
df_test['spam'] = df_test['spam'].replace({'ham': 0, 'spam': 1})

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_data = df_train['text']
train_label = df_train['spam']

test_data = df_test['text']
test_label = df_test['spam']

def get_txt(dat, lab=None) :
  texts = np.asarray(dat)
  labels = np.asarray(lab)

  training_samples = int(len(texts))

  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(texts)
  sequences = tokenizer.texts_to_sequences(texts)

  word_index = tokenizer.word_index

  data = pad_sequences(sequences, maxlen=500) # cut off the words after seeing 500 words in each document(email)
  #print("data shape: ", data.shape)

  np.random.seed(42)
  # shuffle data
  indices = np.arange(data.shape[0])
  np.random.shuffle(indices)
  data = data[indices]
  labels = labels[indices]

  texts_train = data[:training_samples]
  y_train = labels[:training_samples]
  return texts_train, y_train

texts_train , y_train = get_txt(train_data, train_label)
texts_test, y_test = get_txt(test_data, test_label)

In [ ]:

model = Sequential()
# Embedding layer
model.add(Embedding(10000, 32)) #max_features = 10000 / 88584
model.add(SimpleRNN(32))
"""
# Masking layer for pre-trained embeddings
model.add(keras.layers.Masking(mask_value=0.0))
# Recurrent layer

# Fully connected layer
model.add(Dense(32, activation='relu'))
# Dropout for regularization
model.add(Dropout(0.5))
"""
# Output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
# fit the model
history_rnn = model.fit(texts_train, y_train, epochs=10, batch_size=50, validation_split=0.2)

In [ ]:
acc = history_rnn.history['acc']
val_acc = history_rnn.history['val_acc']
loss = history_rnn.history['loss']
val_loss = history_rnn.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, '-', color='orange', label='training acc')
plt.plot(epochs, val_acc, '-', color='blue', label='validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, '-', color='orange', label='training acc')
plt.plot(epochs, val_loss,  '-', color='blue', label='validation acc')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
pred = model.predict_classes(texts_test)
acc = model.evaluate(texts_test, y_test)
proba_rnn = model.predict_proba(texts_test)
from sklearn.metrics import confusion_matrix, accuracy_score
print("Test loss is {0:.2f} accuracy is {1:.2f}  ".format(acc[0],acc[1]))
print(confusion_matrix(pred, y_test))
print('Accuracy : ', accuracy_score(test_label, pred))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def predict_message(pred_text):
  sms_test = [pred_text]
  tokenizer = Tokenizer()
  tokenizer.index_word
  tokenizer.fit_on_texts(sms_test)
  sms_seq = tokenizer.texts_to_sequences(sms_test)
  sms_pad = pad_sequences(sms_seq, maxlen=17, padding='post')
  #sms_test = [pred_text]
  #zero = [1]
  #sms_pad, y = get_txt(sms_test, zero)

  res_ = model.predict(sms_pad)
  res_pred = model.predict_classes(sms_pad)

  if (res_pred == 1 and res_*100 > 94 ) or 1<res_*100 <= 15 or res_*100 < 0.1 :
  #if res_*100 <= 10:
    spam = 'spam'
    prediction = [pred_text, spam]
  else :
    spam = 'ham'
    prediction = [pred_text, spam]
  #print(res_*100, spam)

  return prediction


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    #print(prediction[1])
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
